In [1]:
import os

In [2]:
os.chdir('../')
os.getcwd()

'/home/zakaria/workspace/ai-photo-enhancer'

In [6]:
from src.envs.new_edit_photo import PhotoEditor
from src.envs.photo_env import PhotoEnhancementEnvTest
from src.sac.sac_inference import InferenceAgent
import yaml
import torch

In [7]:
class Config(object):
    def __init__(self, dictionary):
        self.__dict__.update(dictionary)

def load_preprocessor_agent(preprocessor_agent_path, device):
    # with open(
    #     os.path.join(preprocessor_agent_path, "configs/sac_config.yaml")
    # ) as f:
    #     sac_config_dict = yaml.load(f, Loader=yaml.FullLoader)
    with open(
        os.path.join(preprocessor_agent_path, "configs/env_config.yaml")
    ) as f:
        env_config_dict = yaml.load(f, Loader=yaml.FullLoader)
    with open(os.path.join("src/configs/inference_config.yaml")) as f:
        inf_config_dict = yaml.load(f, Loader=yaml.FullLoader)

    inference_config = Config(inf_config_dict)
    # sac_config = Config(sac_config_dict)
    env_config = Config(env_config_dict)

    inference_env = PhotoEnhancementEnvTest(
        batch_size=env_config.train_batch_size,
        imsize=env_config.imsize,
        training_mode=None,
        done_threshold=env_config.threshold_psnr,
        edit_sliders=env_config.sliders_to_use,
        features_size=env_config.features_size,
        discretize=env_config.discretize,
        discretize_step=env_config.discretize_step,
        use_txt_features=(
            env_config.use_txt_features
            if hasattr(env_config, "use_txt_features")
            else False
        ),
        augment_data=False,
        pre_encoding_device=device,
        pre_load_images=False,
        logger=None,
    )

    inference_config.device = device
    preprocessor_agent = InferenceAgent(inference_env, inference_config)
    preprocessor_agent.device = device
    preprocessor_agent.load_backbone(
        os.path.join(preprocessor_agent_path, "models", "backbone.pth")
    )
    preprocessor_agent.load_actor_weights(
        os.path.join(preprocessor_agent_path, "models", "actor_head.pth")
    )
    preprocessor_agent.load_critics_weights(
        os.path.join(preprocessor_agent_path, "models", "qf1_head.pth"),
        os.path.join(preprocessor_agent_path, "models", "qf2_head.pth"),
    )
    return preprocessor_agent